# Импортирование необходимых библиотек

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from PIL import ImageOps
from PIL import Image
import pandas as pd
import numpy as np
import cv2

# Подготовка данных

### Описание датасета:

Данный датасет состоит из двух частей: набор из 14190 изображений прописных русских букв в формате png и текстовый файл, содержащий описание изображений. Текстовый файл включает в себя такую информацию, как: 

- file: название файла, 
- letter: изображенная на картинке буква, 
- label: прядковый номер изображенной буквы в алфавите,
- background: тип фона изображения (где 0 - в полоску, 1 - в клетку, 2 - однотонный, 3 - миллиметровка)


In [2]:
# считываем данные из текстового файла в dataframe
labels1 = pd.read_csv('./data/labels_1650.txt')
labels2 = pd.read_csv('./data/labels_5940.txt')
labels3 = pd.read_csv('./data/labels_6600.txt')

labels = labels1.append(labels2).append(labels3)
labels.head()

,letter,label,file,background
0,а,1,01_01.png,0
1,а,1,01_02.png,0
2,а,1,01_03.png,0
3,а,1,01_04.png,0
4,а,1,01_05.png,0


In [3]:
# выделяем столбец с названиями файлов в отдельную переменную
file_names = labels['file'].tolist()

In [4]:
image_arrays = []

for file in file_names:
    # открываем фото, переводим в ч/б и сжимаем до размера 16х16 пикселей
    img = Image.open(f'./data/letters/{file}')
    img = ImageOps.grayscale(img)
    img = img.resize((16,16))
    
    # приводим фотографию к numpy array и добавляем его в общий список
    img_arr = np.array(img).reshape([1,256])[0]
    image_arrays.append(img_arr)

In [5]:
# добавляем список из фотографий, приведенных в формат np.array, к нашему dataframe'у в качестве столбца 
labels['image_arrays'] = image_arrays

In [6]:
# готовый для дальнейшей работы датасет
labels.head()

,letter,label,file,background,image_arrays
0,а,1,01_01.png,0,"[189, 190, 190, 188, 188, 188, 188, 189, 189, ..."
1,а,1,01_02.png,0,"[175, 176, 175, 174, 173, 173, 174, 174, 173, ..."
2,а,1,01_03.png,0,"[159, 161, 161, 160, 161, 160, 161, 162, 164, ..."
3,а,1,01_04.png,0,"[178, 179, 183, 182, 183, 177, 177, 184, 185, ..."
4,а,1,01_05.png,0,"[146, 147, 148, 150, 151, 153, 155, 157, 156, ..."


In [7]:
# перемешиваем строки в датасете
labels = labels.sample(n=len(labels))

In [8]:
# записываем получившийся dataframe в файл для дальнейшего удобства работы
labels.to_csv(r'./data/labels_full.txt', sep=';')

# Формирование выборок 

In [9]:
# выделяем Х и y
x = labels['image_arrays'].values ###############labels[['image_arrays', 'background']].values
y = labels['letter'].values

In [10]:
# разделяем выборку на обучающую (80%) и тестовую (20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=17)

# Вспомогательные функции

In [11]:
# напишем вспомогательную функцию, выводящую критерии оценки для моделей
def model_metrics(pred, model_name):
    accuracy = round(accuracy_score(y_test, pred) * 100, 2)
    recall = round(recall_score(y_test, pred, average=None).mean() * 100, 2)
    precision = round(precision_score(y_test, pred, average=None).mean() * 100, 2)
    
    accuracies.append(accuracy)
    recalls.append(recall)
    precisions.append(precision)
    model_names.append(model_name)
    
    print('{:=^115s}'.format(f' Критерии оценки для {model_name} '))
    print("{:^115s}".format(f'Precision: {precision}%;    accuracy: {accuracy}%;    recall: {recall}%.'))
    print('='*115)

# Модель №1. Дерево решений. 

In [13]:
# создаем модель классификатор дерева решений
tree_model_name = 'DecisionTreeClassifier'
tree_model = DecisionTreeClassifier(max_depth = 30, random_state = 36)

In [14]:
%%time
# обучаем модель по нашей тренировочной выборке
tree_model.fit(list(x_train), y_train)

CPU times: user 2.16 s, sys: 10.7 ms, total: 2.17 s
Wall time: 2.18 s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=30,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=36, splitter='best')

In [15]:
%%time
# делаем прогноз для отложенной выборки
tree_pred = tree_model.predict(list(x_test))

CPU times: user 4.53 ms, sys: 2.1 ms, total: 6.64 ms
Wall time: 6.25 ms


In [16]:
model_metrics(tree_pred, tree_model_name)

=================================== Критерии оценки для DecisionTreeClassifier ====================================
                            Precision: 15.57%;    accuracy: 15.93%;    recall: 15.41%.                             


# Модель №2. Алгоритм случайного леса. 

In [17]:
# создаем модель
rfc_model_name = 'RandomForestClassifier'
rfc_model = RandomForestClassifier()

In [18]:
%%time 
# обучаем нашу модель на тренировочной выборке
rfc_model.fit(list(x_train), y_train)

/Users/Anastasia/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1.01 s, sys: 21.1 ms, total: 1.03 s
Wall time: 1.04 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
%%time
# делаем прогноз для отложенной выборки
rfc_pred = rfc_model.predict(list(x_test))

CPU times: user 22.8 ms, sys: 14.3 ms, total: 37.2 ms
Wall time: 52.2 ms


In [20]:
model_metrics(rfc_pred, rfc_model_name)

=================================== Критерии оценки для RandomForestClassifier ====================================
                            Precision: 22.98%;    accuracy: 22.51%;    recall: 21.42%.                             


# Модель №3. Логистическая регрессия. 

In [21]:
# создаем модель логистической регрессии
logreg_model_name = 'LogisticRegression'
logreg_model = LogisticRegression(random_state=0)

In [22]:
%%time 
# обучаем нашу модель на тренировочной выборке
logreg_model.fit(list(x_train), y_train)

/Users/Anastasia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Anastasia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CPU times: user 5min 8s, sys: 1.06 s, total: 5min 9s
Wall time: 5min 10s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
%%time
# делаем прогноз для отложенной выборки
logreg_pred = logreg_model.predict(list(x_test))

CPU times: user 16.1 ms, sys: 16.6 ms, total: 32.7 ms
Wall time: 33 ms


In [24]:
model_metrics(logreg_pred, logreg_model_name)

===================================== Критерии оценки для LogisticRegression ======================================
                             Precision: 38.02%;    accuracy: 38.37%;    recall: 37.9%.                             


# Модель №4. Метод ближайших соседей. 

In [25]:
# создаем модель классификатор k ближайших соседей
knn_model_name = 'KNeighborsClassifier'
knn_model = KNeighborsClassifier()

In [26]:
%%time
knn_model.fit(list(x_train), y_train)

CPU times: user 408 ms, sys: 23.1 ms, total: 431 ms
Wall time: 149 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [27]:
%%time
# делаем прогноз для отложенной выборки
knn_pred = knn_model.predict(list(x_test))

CPU times: user 5.91 s, sys: 23 ms, total: 5.94 s
Wall time: 5.96 s


In [28]:
model_metrics(knn_pred, knn_model_name)

===================================== Критерии оценки для LogisticRegression ======================================
                            Precision: 38.96%;    accuracy: 32.54%;    recall: 31.63%.                             


# Модель №5. Метод опорных векторов. 

In [29]:
# создаем модель
svm_model_name = 'SVM'
svm_model = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [30]:
%%time
# обучаем модель
svm_model.fit(list(x_train), y_train)

CPU times: user 41.2 s, sys: 130 ms, total: 41.3 s
Wall time: 40.7 s


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [31]:
%%time
# делаем прогноз для отложенной выборки
svm_pred = svm_model.predict(list(x_test))

CPU times: user 13.1 s, sys: 40 ms, total: 13.2 s
Wall time: 13.2 s


In [32]:
model_metrics(svm_pred, svm_model_name)

============================================= Критерии оценки для SVM =============================================
                             Precision: 48.3%;    accuracy: 38.67%;    recall: 37.0%.                              


# Поиск оптимальных параметров для Модели №5. 

Так как модель метода опорных векторов превзошла все остальные по показателям точности, продолжаем работать с ней. Для повышения точности модели используем метод GridSearchCV, который подберет оптимальные параметры модели. 

In [103]:
# список параметров, которые будем подбирать, и их возможные значения
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [114]:
# создаем модель
grid_model_name = 'GridSearchCV'
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

In [109]:
%%time
# обучаем модель для grid search
grid.fit(list(x_train), y_train)

/Users/Anastasia/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.033, total=  35.3s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.3s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.034, total=  39.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.034, total=  33.6s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.033, total=  33.4s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.034, total=  33.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.034, total=  33.1s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.033, total=  38.6s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.034, total=  38.7s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] ....... C=0.1, gamma=0.01, kernel=rbf, score=0.034, total=  40.8s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] .

[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.034, total=  33.3s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.034, total=  33.4s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV] ......... C=1000, gamma=1, kernel=rbf, score=0.034, total=  33.5s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=1000, gamma=0.1, kernel=rbf, score=0.034, total=  33.4s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=1000, gamma=0.1, kernel=rbf, score=0.034, total=  33.4s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=1000, gamma=0.1, kernel=rbf, score=0.034, total=  33.6s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=0.034, total=  35.3s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 44.6min finished


CPU times: user 45min 25s, sys: 4.73 s, total: 45min 30s
Wall time: 45min 40s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [110]:
# в результате получаем оптимальные параметры нашей модели
grid.best_params_

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

In [111]:
# финальный вид нашей модели - лучшее сочетание параметров
grid.best_estimator_

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [113]:
grid_pred = grid.predict(list(x_test))

In [131]:
model_metrics(grid_pred, grid_model_name)

======================================== Критерии оценки для GridSearchCV =========================================
                            Precision: 45.49%;    accuracy: 41.38%;    recall: 39.39%.                             
